
# ChannelAttentionFusionT — **Satır Satır, Parametre Parametre İnceleme**
Bu notebook, aşağıdaki PyTorch modülünü **en ince ayrıntısına kadar** açıklamak için hazırlanmıştır.

> Hedef: Kodun **her parametresi**, **her satırı**, hatta “küçük” görünen seçimlerin (ör. `bias`, `eps`, `inplace=True`, `register_buffer`, `softmax dim=0` vb.) **neden var olduğunu** ve **ne etkisi olduğunu** anlamak.

---

## İncelenen Kod
Aşağıdaki kod, CBAM tarzı **Channel Attention** mekanizmasının geliştirilmiş bir sürümüdür:
- AvgPool + MaxPool squeeze
- Ortak MLP
- Avg/Max fusion: `sum` veya **öğrenilebilir `softmax`**
- Gate: `sigmoid` veya `hardsigmoid`
- **Temperature** ile “maskenin keskinliği” kontrolü
- İsteğe bağlı **learnable temperature** (pozitif kalması garanti)



In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class ChannelAttentionFusionT(nn.Module):
    def __init__(
        self,
        channels: int,
        reduction: int = 16,
        min_hidden: int = 4,
        fusion: str = "softmax",        # "sum" | "softmax"
        gate: str = "sigmoid",          # "sigmoid" | "hardsigmoid"
        temperature: float = 1.0,
        learnable_temperature: bool = False,
        eps: float = 1e-6,
        act: str = "relu",              # "relu" | "silu"
        bias: bool = True,
        return_fusion_weights: bool = False,
    ):
        super().__init__()
        if fusion not in ("sum", "softmax"):
            raise ValueError("fusion 'sum' veya 'softmax' olmalı.")
        if gate.lower() not in ("sigmoid", "hardsigmoid"):
            raise ValueError("gate 'sigmoid' veya 'hardsigmoid' olmalı.")
        if temperature <= 0:
            raise ValueError("temperature pozitif olmalı.")
        if act not in ("relu", "silu"):
            raise ValueError("act 'relu' veya 'silu' olmalı.")

        self.fusion = fusion
        self.return_fusion_weights = return_fusion_weights
        self.eps = eps

        hidden = max(min_hidden, channels // reduction)

        self.avg = nn.AdaptiveAvgPool2d(1)
        self.mx  = nn.AdaptiveMaxPool2d(1)

        self.fc1 = nn.Conv2d(channels, hidden, 1, bias=bias)
        self.act = nn.ReLU(inplace=True) if act == "relu" else nn.SiLU(inplace=True)
        self.fc2 = nn.Conv2d(hidden, channels, 1, bias=bias)

        if gate.lower() == "sigmoid":
            self.gate_fn = torch.sigmoid
        else:
            self.gate_fn = F.hardsigmoid

        if fusion == "softmax":
            self.fusion_logits = nn.Parameter(torch.zeros(2))
        else:
            self.fusion_logits = None

        self.learnable_temperature = learnable_temperature
        if learnable_temperature:
            t_raw = torch.tensor(float(temperature))
            t_inv = torch.log(torch.exp(t_raw) - 1.0 + eps)
            self.t_raw = nn.Parameter(t_inv)
        else:
            self.register_buffer("T", torch.tensor(float(temperature)))

    def _get_T(self) -> torch.Tensor:
        if self.learnable_temperature:
            return F.softplus(self.t_raw) + self.eps
        return self.T

    def _mlp(self, s: torch.Tensor) -> torch.Tensor:
        return self.fc2(self.act(self.fc1(s)))

    def forward(self, x: torch.Tensor):
        a = self._mlp(self.avg(x))
        m = self._mlp(self.mx(x))

        fusion_w = None
        if self.fusion == "sum":
            z = a + m
        else:
            fusion_w = torch.softmax(self.fusion_logits, dim=0)  # (2,)
            z = fusion_w[0] * a + fusion_w[1] * m

        T = self._get_T()
        ca = self.gate_fn(z / T)  # (B,C,1,1)
        y = x * ca

        if self.return_fusion_weights and (fusion_w is not None):
            return y, ca, fusion_w
        return y, ca


----
----
----
## Cümle özeti aşağıdaki gibidir:

**Öncelikle gerekli parametreleri aldık ve fusion, return_fusion_weights ve eps değişkenlerini self içine kaydettik. Daha sonra hidden kanal sayısını belirledik; bunu yaparken channels // reduction değerinin çok küçük olmaması için max(min_hidden, ...) kullandık. Ardından kanal bilgisini sıkıştırmak için AdaptiveAvgPool2d(1) ve AdaptiveMaxPool2d(1) tanımladık. Sonrasında bu sıkıştırılmış kanal bilgilerini işleyecek olan MLP yapısını kurduk; bunun için önce channels → hidden dönüşümünü yapan fc1 katmanını, ardından aktivasyon fonksiyonunu (ReLU veya SiLU) ve son olarak hidden → channels dönüşümünü yapan fc2 katmanını tanımladık. Daha sonra kanal maskesini üretirken kullanılacak gate_fn fonksiyonunu (sigmoid veya hardsigmoid) belirledik. Sıcaklığın öğrenilebilir olup olmadığına göre iki yol izledik; eğer öğrenilebilir ise verilen temperature değerini başlangıçta korunacak şekilde softplus’ın tersini alarak t_inv hesapladık ve bunu t_raw adlı öğrenilebilir parametre olarak tanımladık, eğer öğrenilebilir değilse temperature değerini register_buffer ile sabit bir değişken olarak modele ekledik. Ardından sıcaklığı güvenli şekilde elde etmek için _get_T fonksiyonunu tanımladık ve kanal özetlerini işlemek için _mlp yardımcı fonksiyonunu yazdık. Forward aşamasında önce girişten avg ve max pooling ile kanal özetlerini çıkardık ve bunları MLP’den geçirerek a ve m değerlerini elde ettik. Eğer fusion yöntemi sum ise bu iki değeri doğrudan topladık, değilse fusion_logits üzerinden softmax alarak modelin avg ve max kaynaklarından hangisine daha fazla güveneceğini öğrendiği ağırlıkları elde ettik ve bu ağırlıklarla a ve m değerlerini birleştirerek z değerini oluşturduk. Daha sonra _get_T fonksiyonu ile sıcaklık değerini aldık, z / T işlemini uygulayıp bunu gate_fn fonksiyonundan geçirerek kanal attention maskesini ca olarak ürettik ve son olarak bu maskeyi giriş tensorü ile çarparak attention uygulanmış çıktı y değerini elde ettik; eğer istenirse analiz amacıyla fusion ağırlıklarını da çıktı olarak döndürdük.**

---
---
---


---
# 1) Bu Modül Ne Yapıyor? (Yüksek Seviye)
Input: `x ∈ R^{B×C×H×W}`  
Output:
- `y ∈ R^{B×C×H×W}` (attention uygulanmış feature)
- `ca ∈ R^{B×C×1×1}` (channel attention maskesi)
- opsiyonel `fusion_w ∈ R^{2}` (avg/max karışım ağırlıkları)

**Kanal attention** şunu yapar:
- Her kanal için bir “önem katsayısı” üretir.
- `ca[:, c, 0, 0]` büyüdükçe o kanal daha çok korunur/boost edilir.
- `y = x * ca` ile her kanal ölçeklenir.

Bu yaklaşım, özellikle CNN backbone’larda “hangi kanalın daha faydalı” olduğunu adaptif seçmeye yarar.

---
# 2) Şekil Takibi (Shape Tracking)
Bir kez netleştirelim:

- `x`: `(B, C, H, W)`
- `avg(x)`: `(B, C, 1, 1)`
- `max(x)`: `(B, C, 1, 1)`
- `MLP(avg(x))`: `(B, C, 1, 1)`
- `MLP(max(x))`: `(B, C, 1, 1)`
- `z`: `(B, C, 1, 1)`
- `ca = gate(z/T)`: `(B, C, 1, 1)`
- `y = x * ca`: `(B, C, H, W)` (broadcast ile)

Bu notasyonu notebook boyunca referans alacağız.



---
# 3) Parametreler: Tek Tek, Ne İşe Yarar?
Aşağıdaki parametrelerin her biri modelin kapasitesini, stabilitesini veya davranışını değiştirir.

## `channels: int`
- Input tensor’ün kanal sayısı: `C`
- Bu modülün “kanal maskesi” de aynı `C` boyutunda üretilir.
- Yanlış verilirse shape uyumsuzluğu çıkar (örn. `Conv2d` ağırlıkları uyuşmaz).

## `reduction: int = 16`
- Bottleneck oranı.
- Hidden boyut yaklaşık `C / reduction` olur.
- Amaç: MLP parametre sayısını düşürmek, overfit/hesap maliyetini azaltmak.

> Örnek: `C=256, reduction=16 => hidden≈16`

## `min_hidden: int = 4`
- Çok küçük `C` değerlerinde `C//reduction` sıfıra yaklaşabilir.
- `min_hidden`, hidden boyutunu “en az şu kadar” yapar.
- Bu, modülün “çalışabilir” kalmasını garanti eder.

## `fusion: str = "softmax"`  (`"sum"` | `"softmax"`)
- Avg ve Max squeeze sonuçlarının nasıl birleştirileceğini seçer.

### `fusion="sum"`
- Klasik CBAM davranışı: `z = a + m`
- Avg ve Max aynı ağırlıkla (1,1) katkı verir.
- Öğrenilebilir değildir.

### `fusion="softmax"`
- İki learnable logit vardır: `fusion_logits = [l_avg, l_max]`
- `fusion_w = softmax(fusion_logits)` ile `w_avg+w_max=1`
- `z = w_avg*a + w_max*m`
- Model dataset’e göre “avg mi max mı daha iyi?” öğrenebilir.

**Önemli nüans:** Bu ağırlıklar **global** (sample’a göre değil). Yani her input için aynı `fusion_w` kullanılır.

## `gate: str = "sigmoid"` (`"sigmoid"` | `"hardsigmoid"`)
- Son maske üretirken kullanılan aktivasyon.
- `sigmoid`: daha yumuşak, standart, türevleri düzgün.
- `hardsigmoid`: daha hızlı, bazı donanımlarda daha verimli, saturasyon davranışı farklı.

> `hardsigmoid` özellikle mobil/edge modellerde tercih edilebilir.

## `temperature: float = 1.0`
- `z` logitlerinin gate’e girmeden önce ölçeklenmesi: `z / T`
- `T` küçükse => daha “sert” maskeler (0/1’e yaklaşır)
- `T` büyükse => daha “yumuşak” maskeler (0.5 civarına yaklaşır)

## `learnable_temperature: bool = False`
- `T` sabit mi olacak, öğrenilecek mi?
- `False`: `T` buffer olarak sabit tutulur.
- `True`: `T` bir parametre olur ama **pozitif kalması garanti** edilir (softplus ile).

## `eps: float = 1e-6`
- Sayısal stabilite için.
- Öğrenilebilir temperature ters dönüşümünde ve softplus sonrası eklenir.
- Amaç: `log(0)` gibi patlamaları engellemek, `T=0` olmasını önlemek.

## `act: str = "relu"` (`"relu"` | `"silu"`)
- MLP içindeki nonlinearity.
- `relu`: hızlı, yaygın
- `silu`: daha pürüzsüz, bazen daha iyi performans (özellikle modern convnet’lerde)

## `bias: bool = True`
- `fc1` ve `fc2` 1×1 conv katmanlarında bias kullanılsın mı?
- BN/LN gibi normalizasyon yoksa bias bazen faydalı olabilir.
- Ama bazı tasarımlarda bias gereksiz parametre olur (özellikle sonraki norm katmanı varsa).

## `return_fusion_weights: bool = False`
- Debug/analiz için.
- `fusion="softmax"` iken `fusion_w` döndürmek isteyebilirsin.
- Eğitim sırasında genelde kapalı tutulur (API sadeleşir).



---
# 4) “Virgülüne Kadar” Kod Yapısı ve Python İmzaları
Bu bölüm, küçük görünen ama önemli olan dil/uygulama detaylarını açıklar.

## `def __init__( self, ... , ):`
- Parametre listesinin sonunda virgül olması (PEP8):
  - Diff’lerde satır ekleme/çıkarma kolaylaşır.
  - Otomatik formatter’larla uyumludur.
  - Hiçbir runtime etkisi yoktur; bakım kolaylığı sağlar.

## Type hints: `channels: int`
- PyTorch için zorunlu değil.
- Okunabilirlik ve IDE autocompletion için faydalıdır.
- MyPy gibi statik analiz araçlarıyla hata yakalamayı sağlar.

## `super().__init__()`
- `nn.Module` iç yapısını kurar:
  - Parameter/buffer kaydı
  - `state_dict` mekanizması
  - `.to(device)` / `.cuda()` davranışları
- Bunu çağırmazsan modülün parametreleri düzgün kayıt olmaz.

---
# 5) Girdi Validasyonları: Neden Bu `if` Kontrolleri Var?
```python
if fusion not in ("sum","softmax"): raise ValueError(...)
```
Bu kontroller:
- Sessiz bug’ları önler (örn. `fusion="sofmax"` typo).
- Hata çıktısını daha anlaşılır yapar.
- Deney/ablasyon yaparken yanlış konfigürasyonu anında yakalar.

`temperature <= 0` kontrolü kritiktir:
- `z / T` bölümünde `T=0` tanımsız
- Negatif T maskeyi ters ölçekler ve yorumlanabilirliği bozar.



---
# 6) `hidden = max(min_hidden, channels // reduction)` Satırı
Bu satır, MLP’nin orta katman kanal sayısını belirler.

## Neden bottleneck var?
- Kanal attention MLP’si normalde `C -> C/r -> C`
- `C` büyükse doğrudan `C->C` dense çok pahalı olur.
- Bottleneck, parametre ve MAC’i düşürür.

## Neden `//` (integer division)?
- PyTorch `Conv2d` kanal sayısı integer olmalı.
- `channels/reduction` float üretmesin diye `//` kullanılır.

## Neden `max(min_hidden, ...)`?
- `C=8`, `reduction=16` olursa `8//16=0` çıkar.
- 0 kanal diye bir şey olamaz.
- `min_hidden` bunu engeller ve “en az 4” gibi bir taban verir.

**Pratik öneri:**  
Küçük backbone katmanlarında (`C=16/32`) `min_hidden` özellikle önemlidir.



---
# 7) `AdaptiveAvgPool2d(1)` ve `AdaptiveMaxPool2d(1)` Ne Demek?
```python
self.avg = nn.AdaptiveAvgPool2d(1)
self.mx  = nn.AdaptiveMaxPool2d(1)
```
Bu katmanlar, `H×W` boyutunu **1×1**’e indirir.

## Adaptive ne sağlar?
- Girdi boyutu değişse bile (ör. 56×56, 80×80) çıkış her zaman 1×1 olur.
- Bu sayede blok “resolution-agnostic” çalışır.

## Avg vs Max farkı
- Avg: kanalın genel dağılımını ölçer (smooth/istikrarlı).
- Max: kanalın en güçlü aktivasyonunu yakalar (spike/edge/trigger).

CBAM’in fikri: İkisi birlikte daha iyi sinyal verir.



---
# 8) MLP Neden `Conv2d(1×1)` ile Yazılmış?
```python
self.fc1 = nn.Conv2d(channels, hidden, 1, bias=bias)
self.fc2 = nn.Conv2d(hidden, channels, 1, bias=bias)
```
Bu kullanım, SE/CBAM literatüründeki klasik yaklaşımdır.

## Neden Linear değil?
- Çünkü tensor shape’i `(B,C,1,1)` ve Conv2d bunu doğal işler.
- Linear kullanırsan reshape/squeeze gerekir.
- Conv2d ile PyTorch graph’ı daha “standart CNN” gibi kalır.

## Parametre sayısı (yaklaşık)
- `fc1`: `C*hidden` (+ bias varsa `hidden`)
- `fc2`: `hidden*C` (+ bias varsa `C`)

Toplam yaklaşık `2*C*hidden` (bias ihmal).



---
# 9) Aktivasyon: `ReLU(inplace=True)` vs `SiLU(inplace=True)`
```python
self.act = nn.ReLU(inplace=True) if act=="relu" else nn.SiLU(inplace=True)
```
## `inplace=True` ne yapar?
- Aktivasyonu aynı tensor üzerinde uygular (ek memory tahsisini azaltabilir).
- Bazı durumlarda autograd ile çakışma riskleri olabilir ama burada tipik kullanımda sorun çıkmaz.

## ReLU vs SiLU
- ReLU: hızlı, sparse aktivasyon, klasik.
- SiLU (Swish): pürüzsüz, küçük negatiflerde gradient akışı sağlar.
- Modern CNN’lerde (EfficientNet, ConvNeXt türevleri) SiLU sık tercih edilir.

Bu seçimin dikkat modülünde etkisi:
- MLP’nin ürettiği logits’in dağılımını değiştirir.
- Dolaylı olarak maskenin keskinliğini/kalibrasyonunu etkileyebilir.



---
# 10) Gate Fonksiyonu Seçimi: `sigmoid` vs `hardsigmoid`
```python
if gate.lower() == "sigmoid":
    self.gate_fn = torch.sigmoid
else:
    self.gate_fn = F.hardsigmoid
```
## Neden fonksiyonu attribute olarak saklıyoruz?
- Forward içinde `if` yapmaktan kaçınır (micro-opt, okunabilirlik).
- Modül davranışı init’te “kilitlenir”.

## `torch.sigmoid` vs `F.hardsigmoid`
- Sigmoid: `1/(1+exp(-x))` (smooth)
- HardSigmoid: parçalı doğrusal approx (daha hızlı)

HardSigmoid etkisi:
- Saturation bölgeleri farklıdır.
- Bazı donanımlarda hız avantajı olabilir.



---
# 11) Fusion Mantığı: `sum` ve `softmax` Derin Analiz
Bu bloktaki asıl “twist” burası.

## 11.1) `fusion_logits = nn.Parameter(torch.zeros(2))`
- İki scalar parametre.
- Başlangıç: `[0, 0]`

## 11.2) `softmax(..., dim=0)` neden `dim=0`?
`fusion_logits` shape’i `(2,)` olduğu için:
- `dim=0` tek boyut üzerinde softmax demek.
- Sonuç yine `(2,)` olur ve toplamı 1’dir.

## 11.3) Başlangıçta neden zeros?
- `softmax([0,0]) = [0.5, 0.5]`
- Yani “avg ve max eşit” ile başlıyor.
- Eğitim, hangi taraf daha faydalıysa oraya kaydırıyor.

## 11.4) Bu fusion “global”dır
Dikkat: `fusion_logits` **input’a bağlı değil**.
- Her batch, her görüntü için aynı `fusion_w` kullanılır.
- Bu bir “model seviyesinde hiperparametreyi öğrenme” gibi çalışır.

Eğer input’a bağlı olmasını isteseydin:
- `fusion_logits`’ı bir router ile `avg(x), max(x)`’ten üretirdin.

## 11.5) `sum` neden hala var?
Ablation / baseline için:
- softmax fusion bazı modellerde gereksiz ek parametre olabilir.
- `sum` daha basit ve bazen daha stabil.



---
# 12) Temperature: Matematiksel ve Pratik Etki
```python
ca = gate_fn(z / T)
```
Burada `z` gate’e giren logit (B,C,1,1).

## 12.1) T küçülürse
- `z/T` büyür (mutlak değer)
- sigmoid daha hızlı 0/1’e gider
- maske **keskinleşir**

## 12.2) T büyürse
- `z/T` küçülür
- sigmoid 0.5 civarında kalır
- maske **yumuşar** (daha az agresif)

Bu, bir “kalibrasyon / confidence” kontrolüdür.

## 12.3) Neden learnable temperature?
Her dataset ve her backbone katmanı için ideal keskinlik farklı olabilir.
- Erken katmanlarda çok keskin maskeler bilgi kaybettirebilir.
- Geç katmanlarda keskin maskeler faydalı olabilir.

Learnable T ile model bunu ayarlayabilir.



---
# 13) Learnable Temperature’ın İnşa Edilmesi (En İnce Detay)
Kod:
```python
t_raw = torch.tensor(float(temperature))
t_inv = torch.log(torch.exp(t_raw) - 1.0 + eps)
self.t_raw = nn.Parameter(t_inv)

# kullanım
T = softplus(t_raw) + eps
```
## 13.1) Neden `softplus`?
`softplus(x) = log(1+exp(x))` her zaman pozitiftir.
- Böylece `T > 0` garanti.

## 13.2) Neden “inverse softplus” gibi bir şey yapılıyor?
İstenilen: Eğitim başında `T` tam olarak `temperature` olsun.

Eğer direkt `self.t_raw = nn.Parameter(torch.tensor(temperature))` yapsaydın:
- `_get_T()` softplus uygulayacağı için başlangıç `T` ≠ temperature olurdu.

Bu yüzden:
- `t_inv` seçiliyor ki `softplus(t_inv) ≈ temperature` olsun.

Yaklaşık ters:
- `softplus^{-1}(t) = log(exp(t) - 1)`
- Buradaki `+ eps` sayısal stabilite için.

## 13.3) `eps` burada niye kritik?
`temperature` çok küçükse `exp(t)-1` ~ 0 olabilir.
- `log(0)` → `-inf`
- `+ eps` bunu önler.

## 13.4) Learnable değilse `register_buffer("T", ...)`
- Bu bir Parameter değildir, gradient almaz.
- Ama `state_dict`’e girer.
- `.to(device)` ile GPU/CPU arasında taşınır.


### İşte LOG ve −1 buradan geliyor
Adım adım (çok basit):

1️⃣
```python
log(1 + exp(t_inv)) = temperature
```

2️⃣
Log’dan kurtulmak için exp al:
```python
1 + exp(t_inv) = exp(temperature)
```


3️⃣
1’i karşıya at:
```python
exp(t_inv) = exp(temperature) - 1
```

4️⃣
Şimdi t_inv’i yalnız bırakmak için log al:
```python
t_inv = log(exp(temperature) - 1)
```


🔥 İşte bu yüzden log var

🔥 İşte bu yüzden −1 var

* Başka hiçbir numara yok.
* Bu sadece softplus’ın tersini almak.

Bu, sabit hyperparametreyi “modelin parçası” gibi tutmanın temiz yoludur.



---
# 14) Yardımcı Metodlar: `_get_T()` ve `_mlp()` Neden Var?
## `_get_T()`
- Learnable/sabit T ayrımını tek yerde toplar.
- Forward’ı temiz tutar.

## `_mlp(s)`
```python
return fc2(act(fc1(s)))
```
- Avg ve Max squeeze çıktıları aynı MLP’den geçiyor.
- Bu “shared MLP” CBAM’in standardıdır.
- Kod tekrarını azaltır.

**Not:** Shared MLP demek:
- Avg ve Max aynı ağırlıkları paylaşır.
- Böylece model iki squeeze’i aynı “ölçekte” kıyaslar.



---
# 15) Forward: Satır Satır, Ne Oluyor?
```python
a = _mlp(avg(x))
m = _mlp(mx(x))
```
1) `avg(x)` ile her kanalın ortalaması alınır → `(B,C,1,1)`  
2) MLP ile kanal logitleri üretilir → `(B,C,1,1)`  
3) Aynısı max için yapılır.

```python
if fusion=="sum": z=a+m
else:
    fusion_w = softmax(fusion_logits)  # (2,)
    z = fusion_w[0]*a + fusion_w[1]*m
```
4) İki logit kaynağı birleştirilir.
- sum: sabit
- softmax: öğrenilebilir karışım

```python
T = _get_T()
ca = gate_fn(z/T)
```
5) Temperature ile ölçekleme yapılır.
6) Gate fonksiyonu ile 0..1 aralığına sıkıştırılır.

```python
y = x * ca
```
7) Broadcast çarpımı:
- `ca` her kanal için tek katsayıdır.
- `H×W` boyunca uygulanır.

Return:
- Normalde `(y, ca)`
- Debug modunda `(y, ca, fusion_w)`



---
# 16) Köşe Durumlar ve Hata Senaryoları (Bu Kod Neyi Önler?)
## 16.1) `channels // reduction == 0`
- `min_hidden` ile önlenir.

## 16.2) `temperature <= 0`
- bölme hatası / anlamsız ölçekleme
- init’te engellenir.

## 16.3) `fusion="softmax"` ama `return_fusion_weights=True` değil
- `fusion_w` hesaplanır ama döndürülmez.
- Bu “debug” için gereksiz çıktı yaratmamak adına normaldir.

## 16.4) `fusion="sum"` iken `return_fusion_weights=True`
- `fusion_w` None kalır, döndürülmez.
- Kod bunu zaten kontrol ediyor: `(fusion_w is not None)`



---
# 17) Mini Deneyler: Şekil Kontrolü, Fusion Ağırlıkları, Temperature Etkisi
Aşağıdaki hücreler:
- Modülü instantiate eder
- Output shape’lerini doğrular
- `fusion_w` nasıl değişiyor gösterir
- Temperature maskeyi nasıl keskinleştiriyor görselleştirir (istatistiksel)

> Not: Bu hücreler eğitim yapmaz, sadece davranışı gözlemler.


In [3]:

# 17.1) Basit shape testi
torch.manual_seed(0)
x = torch.randn(2, 64, 56, 56)

m_sum = ChannelAttentionFusionT(64, fusion="sum", return_fusion_weights=True)
y_sum, ca_sum = m_sum(x)
print("sum:", y_sum.shape, ca_sum.shape)

m_soft = ChannelAttentionFusionT(64, fusion="softmax", return_fusion_weights=True)
y_soft, ca_soft, fw = m_soft(x)
print("softmax:", y_soft.shape, ca_soft.shape, fw.shape, fw)


sum: torch.Size([2, 64, 56, 56]) torch.Size([2, 64, 1, 1])
softmax: torch.Size([2, 64, 56, 56]) torch.Size([2, 64, 1, 1]) torch.Size([2]) tensor([0.5000, 0.5000], grad_fn=<SoftmaxBackward0>)


In [4]:

# 17.2) Temperature etkisi: maskenin dağılımına bak
def stats(t):
    m = ChannelAttentionFusionT(64, fusion="sum", temperature=t, learnable_temperature=False)
    y, ca = m(x)
    return {
        "T": t,
        "ca_mean": float(ca.mean()),
        "ca_std": float(ca.std()),
        "ca_min": float(ca.min()),
        "ca_max": float(ca.max()),
    }

for t in [0.5, 1.0, 2.0, 5.0]:
    print(stats(t))


{'T': 0.5, 'ca_mean': 0.5130427479743958, 'ca_std': 0.29812273383140564, 'ca_min': 0.04223500192165375, 'ca_max': 0.9689000844955444}
{'T': 1.0, 'ca_mean': 0.4672200679779053, 'ca_std': 0.21377702057361603, 'ca_min': 0.10203836113214493, 'ca_max': 0.8694762587547302}
{'T': 2.0, 'ca_mean': 0.5006216764450073, 'ca_std': 0.13165560364723206, 'ca_min': 0.21391931176185608, 'ca_max': 0.8047383427619934}
{'T': 5.0, 'ca_mean': 0.5045205950737, 'ca_std': 0.027636060491204262, 'ca_min': 0.45041096210479736, 'ca_max': 0.549249529838562}


C:\Users\hdgn5\AppData\Local\Temp\ipykernel_22232\1634033655.py:7: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:837.)
  "ca_mean": float(ca.mean()),


In [5]:

# 17.3) Fusion logits'i elle oynatınca fusion_w nasıl değişir?
m = ChannelAttentionFusionT(64, fusion="softmax", return_fusion_weights=True)
with torch.no_grad():
    m.fusion_logits[:] = torch.tensor([2.0, 0.0])  # avg ağırlığını artır
y, ca, fw = m(x)
print("logits=[2,0] => fusion_w:", fw)

with torch.no_grad():
    m.fusion_logits[:] = torch.tensor([0.0, 2.0])  # max ağırlığını artır
y, ca, fw = m(x)
print("logits=[0,2] => fusion_w:", fw)


logits=[2,0] => fusion_w: tensor([0.8808, 0.1192], grad_fn=<SoftmaxBackward0>)
logits=[0,2] => fusion_w: tensor([0.1192, 0.8808], grad_fn=<SoftmaxBackward0>)


In [6]:

# 17.4) Learnable temperature gerçekten pozitif mi? Başlangıçta temperature'a eşit mi?
m = ChannelAttentionFusionT(64, fusion="sum", temperature=1.7, learnable_temperature=True)
T0 = m._get_T().item()
print("Başlangıç T:", T0, "(hedef 1.7 civarı)")

# Parametreyi değiştirince T nasıl değişiyor?
with torch.no_grad():
    m.t_raw += 1.0
print("t_raw artırınca T:", m._get_T().item())

with torch.no_grad():
    m.t_raw -= 10.0
print("t_raw çok azaltınca T (hala >0 olmalı):", m._get_T().item())


Başlangıç T: 1.7000011205673218 (hedef 1.7 civarı)
t_raw artırınca T: 2.5772933959960938
t_raw çok azaltınca T (hala >0 olmalı): 0.0005529767950065434



---
# 18) Pratik Kullanım Notları (Backbone/YOLO için)
- **Early layers** (yüksek çözünürlük, düşük semantik): Çok keskin attention zarar verebilir → `temperature` biraz daha büyük denenebilir.
- **Late layers** (düşük çözünürlük, yüksek semantik): Keskin attention faydalı olabilir → `temperature` küçük denenebilir.
- `fusion="softmax"` genelde küçük bir ek parametreyle “hangi squeeze daha iyi?”yi öğrenir; ablation için `sum` ile kıyaslamak mantıklıdır.
- `hardsigmoid` hız odaklı olabilir; `sigmoid` daha standart ve daha pürüzsüz gradient verir.

> En iyi ayar deneysel: backbone, veri, optimizasyon ve batch size ile değişir.



---
# 19) Ek: Parametre Sayısı Hesabı (Yaklaşık)
`hidden = max(min_hidden, C//reduction)`

- `fc1`: `C*hidden` (+ bias varsa `hidden`)
- `fc2`: `hidden*C` (+ bias varsa `C`)
Toplam: `2*C*hidden + (hidden + C if bias else 0)`

Fusion:
- `fusion="softmax"` ise +2 parametre (`fusion_logits`)

Learnable temperature:
- `learnable_temperature=True` ise +1 parametre (`t_raw`)

Bu blok çok ucuz kalır ama etkisi büyük olabilir.
